### 1. Import Packages

In [47]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

All packages successfully loaded


### 2. Load Data & Peak Sheet

In [48]:
df = pd.read_excel("/Users/nova/Desktop/Thesis/Metabolomics ML/data/MTBLS92.xlsx", index_col="Idx")
df2 = df[(df.Class == 1) | (df.Class == 0)]
df3 = df2.drop(columns=['SampleID', 'Timepoint', 'Series', 'Chemotherapy response', 'Grade'])

### 2.1 Missing columns extraction

In [49]:
missing_columns = df3.columns[df3.isnull().any()]
print(missing_columns)

Index(['M123'], dtype='object')


Le feature con valori mancanti sono: ['Grade', 'Chemotherapy response', 'M123']. Tuttavia, l'unica feature che ci interessa è il metabolita M123, ovvero TG(50:5).

### 2.2 Data partitioning

In [50]:
# Estraiamo le righe con dati mancanti nella colonna 'M123'
data_missing = df3[df3['M123'].isnull()]

# Estraiamo le righe senza dati mancanti nella colonna 'M123'
data_not_missing = df3.dropna(subset=['M123'])

### 2.3 Building machine learning model for imputation

In [51]:
# Separiamo le features (X) dalla variabile di destinazione (y)
X = data_not_missing.drop(columns=['M123'])
y = data_not_missing['M123']

# Dividiamo il dataset in set di training e di test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creiamo e alleniamo il modello
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

### 2.4 Imputation

In [52]:
# Estraiamo le features dalla prima parte del dataset
X_missing = data_missing.drop(columns=['M123'])

# Facciamo le previsioni per i dati mancanti
predicted_values = model.predict(X_missing)

# Sostituiamo i dati mancanti con le previsioni
data_missing['M123'] = predicted_values

# Combiniamo nuovamente le parti del dataset
dataset_completo = pd.concat([data_not_missing, data_missing])

/var/folders/dv/cqkd8q211lb6w_8jd__s86d40000gn/T/ipykernel_67608/261185541.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_missing['M123'] = predicted_values


### 3. Extract X & Y

In [53]:
X = dataset_completo.drop(columns=['Class'])
y = dataset_completo.Class
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4. Hyperparameters optimization

### 5. Build Model

In [54]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print("Accuracy: ", model.score(X_test, y_test))

Accuracy:  0.6862745098039216
